In [1]:
import numpy as np
import random
import json

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [3]:
from nltk_utils import bag_of_words, stem, tokenize
from model import NeuralNet

[nltk_data] Downloading package punkt to /Users/aaronwong/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
with open('intents.json', 'r') as f:
    intents = json.load(f)
    
print(intents)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'Hey', 'How are you', 'Is anyone there?', 'Hello', 'Good day'], 'responses': ['Hey :-)', 'Hello, thanks for visiting', 'Hi there, what can I do for you?', 'Hi there, how can I help?']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye'], 'responses': ['See you later, thanks for visiting', 'Have a nice day', 'Bye! Come back again soon.']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", "Thank's a lot!"], 'responses': ['Happy to help!', 'Any time!', 'My pleasure']}, {'tag': 'items', 'patterns': ['Which items do you have?', 'What kinds of items are there?', 'What do you sell?'], 'responses': ['We sell coffee and tea', 'We have coffee and tea']}, {'tag': 'payments', 'patterns': ['Do you take credit cards?', 'Do you accept Mastercard?', 'Can I pay with Paypal?', 'Are you cash only?'], 'responses': ['We accept VISA, Mastercard and Paypal', 'We accept most major credit cards, and Paypal']}, {'tag': 'd

In [5]:
all_words = []
tags = []
xy = []

In [6]:
for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        w = tokenize(pattern)
        all_words.extend(w)
        xy.append((w, tag))

In [7]:
ignore_words = ['?', '!', '.', ',']

In [8]:
all_words = [stem(w) for w in all_words if w not in ignore_words]

In [9]:
all_words = sorted(set(all_words))
tags = sorted(set(tags))

In [10]:
X_train = []
Y_train = []

In [11]:
for (sentence, tag) in xy:
    bag = bag_of_words(sentence, all_words)
    X_train.append(bag)
    label = tags.index(tag)
    Y_train.append(label)

In [12]:
X_train = np.array(X_train)
Y_train = np.array(Y_train)

In [13]:
class ChatDataset(Dataset):
    def __init__(self):
        self.num_samples = len(X_train)
        self.X_data = X_train
        self.Y_data = Y_train
    
    def __getitem__(self, i):
        return self.X_data[i], self.Y_data[i]
    
    def __len__(self):
        return self.num_samples
    

In [14]:
# Hyperparamters
epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(all_words)
hidden_size = 8
output_size = len(tags)

In [15]:
dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=0)

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [17]:
model = NeuralNet(input_size, hidden_size, output_size).to(device)

In [18]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [19]:
for epoch in range(epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(device)
        
        # forward
        outputs = model(words)
        loss = criterion(outputs, labels)
        
        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch + 1) % 100 == 0:
        print(f'epoch {epoch + 1}/{epochs}, loss={loss.item():.4f}')
        
print(f'final loss, loss={loss.item():.4f}')

epoch 100/1000, loss=0.8986
epoch 200/1000, loss=0.0965
epoch 300/1000, loss=0.0265
epoch 400/1000, loss=0.0035
epoch 500/1000, loss=0.0028
epoch 600/1000, loss=0.0027
epoch 700/1000, loss=0.0009
epoch 800/1000, loss=0.0030
epoch 900/1000, loss=0.0004
epoch 1000/1000, loss=0.0005
final loss, loss=0.0005


In [20]:
data = {
    "model_state": model.state_dict(),
    "input_size": input_size,
    "output_size": output_size,
    "hidden_size": hidden_size,
    "all_words": all_words,
    "tags": tags
}

In [21]:
FILE = "data.pth"

In [22]:
torch.save(data, FILE)

In [23]:
print(f'training complete and file saved to {FILE}')

training complete and file saved to data.pth
